## Week 3: Non-ML Approaches to Remote Sensing

### Lecture
Slides: https://docs.google.com/presentation/d/e/2PACX-1vTOTYqwvctmoNefZbcLZhek4vaHxzxiqc4KJCjrBGV1-TrTWnoKF3SMTmdE3-mcEzAKqkZxNuO3DT7f/pub?start=false&loop=false&delayms=3000

### Additional Resources
Applied Remote Sensing Training Program, NASA, https://www.earthdata.nasa.gov/data/projects/arset/learn 

Google Earth Engine Tutorials, https://google-earth-engine.com/ 

### Lab: Wildfire Detection and Burn Severity Assessment

In this lab, you'll analyze a real wildfire event using techniques non-ML techniques, including indices and change detection. These methods form the foundation of remote sensing analysis and remain widely used in practice.

You will not be given any code. Instead, you will work through tutorials developed by ESA's Earth Observation Processing Framework (EOPF) team and adapt their code to complete the tasks below.

**Study area**: Province of Nuoro, Sardinia, Italy  
**Event**: Wildfire, June 10, 2025 (~1000 hectares burned)  
**Dataset**: Sentinel-2 L2A

#### Reference Materials

_Note: the EoPF toolikit is under active development and is, as of Jan. 29, 2026, not rendering properly. The text and code still remain accessible, though. For our tutorial, you'll want to make the following modifications:
- Use only the first `search_box` as your AOI, not the larger `bbox_vis` that they define later.
- Access the Sentinel 2 data via Microsoft Planetary Computer, not EoPF's Zarr store, since the latter is not yet functional._

These tutorials contain everything you need. You don't need to work through them linearly; pull what you need for each task.

- [Fire in Sardinia - Part 1](https://eopf-toolkit.github.io/eopf-101/06_eopf_zarr_in_action/61_sardinia_s2_tfci.html): True and false color composites, pre/post comparison
- [Fire in Sardinia - Part 3](https://eopf-toolkit.github.io/eopf-101/06_eopf_zarr_in_action/63_sardinia_dNBR.html): Normalized Burn Ratio (NBR) and burn severity classification

For the time series section, this Project Pythia tutorial demonstrates useful visualization patterns:

- [Project Pythia: NDMI Time Series](https://projectpythia.org/landsat-ml-cookbook/notebooks/ndmi/)

In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pystac_client
import planetary_computer
import odc.stac
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster
from IPython.display import Image
from shapely.geometry import box
from odc.stac import stac_load
from skimage import exposure
from zarr_wf_utils import (
    validate_scl,
    normalisation_str_gm
)

#### Part 1: Pre-Fire Baseline

1. Connect to the Microsoft Planetary Computer STAC catalog and search for Sentinel-2 L2A imagery over the study area for June 3, 2025 (one week before the fire).

2. Open the retrieved item and explore its structure. What groups are available? Where are the spectral bands stored? Plot a rendered preview, as we did in the previous week's lab.

3. Create a cloud mask using the Scene Classification Layer (SCL) band. What pixel types does this mask remove, and why is this necessary?

4. Produce a **true color composite** (RGB) for the pre-fire date. Make sure to normalize the input, per the EoPF toolkit. Apply histogram equalization to improve visual contrast. Compare the un-equalized and equalized images. What differences do you observe?

5. Produce a **false color composite** using SWIR (B12), NIR (B8A), and Blue (B02). How does vegetation appear in this band combination?


In [ ]:
# Open the Planetary Computer STAC catalog.
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                    modifier=planetary_computer.sign_inplace)
# Print the catalog title.
catalog.title

In [ ]:
# Get the Sentinel-2 Level 2A collection.
collection = "sentinel-2-l2a"
sentinel_collection = catalog.get_collection(collection)

In [ ]:
sentinel_collection

In [ ]:
# Create a Dask LocalCluster.
dask_cluster = LocalCluster(
    processes = True,
    n_workers = 2,
    threads_per_worker = 2,
    memory_limit = "8GB"
)

client = Client(dask_cluster)

# Configure ODC STAC for cloud-optimized access.
odc.stac.configure_rio(cloud_defaults = True, client = client)

In [ ]:
# Bounding box for province of Nuoro, Sardinia, Italy.
bbox = [9.239709, 40.290681, 9.396964, 40.351409] # min lon, min lat, max lon, max lat
pre_datetime = "2025-06-01T00:00:00Z/2025-06-09T23:59:59Z"
post_datetime = "2025-07-11T00:00:00Z/2025-07-19T23:59:59Z"

In [ ]:
# Perform the pre-fire search.
pre_search = catalog.search(
    bbox = bbox,
    datetime = pre_datetime,
    collections = collection
    )

pre_search

In [ ]:
# Perform the post-fire search.
post_search = catalog.search(
    bbox = bbox,
    datetime = post_datetime,
    collections = collection
    )

post_search

In [ ]:
# Get the items from the pre-fire search.
pre_items = pre_search.item_collection()
print(f"Returned {len(pre_items)} items.")

In [ ]:
# Get the items from the post-fire search.
post_items = post_search.item_collection()
print(f"Returned {len(post_items)} items.")

In [ ]:
# Create a dictionary mapping pre-fire item indices and IDs.
pre_item_id = {(i, item.id): i for i, item in enumerate(pre_items)}
pre_item_id

In [ ]:
# Create a dictionary mapping post-fire item indices and IDs.
post_item_id = {(i, item.id): i for i, item in enumerate(post_items)}
post_item_id

In [ ]:
# Plot rendered preview with footprint.
pre_selected_item = pre_items[0]
pre_selected_item

In [ ]:
pre_selected_item.assets

In [ ]:
Image(url = pre_selected_item.assets["rendered_preview"].href)

In [ ]:
df = gpd.GeoDataFrame.from_features(pre_items.to_dict(), crs = "epsg:4326")

bbox_geom = box(*bbox)
bbox_gdf = gpd.GeoDataFrame([{"geometry": bbox_geom}], crs = "epsg:4326")

# Create map with footprints.
map_preview = df[["geometry", "datetime", "eo:cloud_cover"]].explore(
    column = "eo:cloud_cover", style_kwds = {"fillOpacity": 0.1}
)

# Add bbox polygon on top.
bbox_gdf.explore(
    m = map_preview, style_kwds = {"color": "red", "weight": 3, "fillOpacity": 0}, name = "Bounding Box"
)

map_preview

In [ ]:
# Load pre-fire data including qa_pixel
pre_data = stac_load(
    pre_items,
    bands = ["B12", "B8A", "B04", "B03", "B02", "SCL"],
    crs = "EPSG:3857",
    resolution = 10,
    chunks = {"time": -1, "x": 64, "y": 64},
    bbox = bbox,
)

In [ ]:
pre_data

In [ ]:
# Load post-fire data including qa_pixel
post_data = stac_load(
    post_items,
    bands = ["B12", "B8A", "B04", "B03", "B02", "SCL"],
    crs = "EPSG:3857",
    resolution = 10,
    chunks = {"time": -1, "x": 128, "y": 128},
    bbox = bbox,
)

In [ ]:
post_data

In [ ]:
total_bytes = sum(pre_data[var].nbytes for var in pre_data.data_vars)
print(f"Total size: {total_bytes / 1e9:.2f} GB ({total_bytes / 1e6:.2f} MB)")

In [ ]:
total_bytes = sum(post_data[var].nbytes for var in post_data.data_vars)
print(f"Total size: {total_bytes / 1e9:.2f} GB ({total_bytes / 1e6:.2f} MB)")

In [ ]:
print(f"CRS: {pre_data.spatial_ref.spatial_ref}")
print(f"Spatial ref attrs: {pre_data.spatial_ref.attrs}")

In [ ]:
print(f"CRS: {post_data.spatial_ref.spatial_ref}")
print(f"Spatial ref attrs: {post_data.spatial_ref.attrs}")

In [ ]:
# Create cloud mask from scl.
# Mask is True where pixels are CLEAR.
scl = pre_data["SCL"].squeeze()
scl_mask = scl.isin([2, 4, 5, 6, 7, 11])

# Apply mask to spectral bands (SCL become NaN).
pre_data_masked = pre_data[["B12", "B8A", "B04", "B03", "B02"]].where(scl_mask)

# Median now ignores NaN (SCL) pixels.
pre_median = pre_data_masked.median(dim = "time")
pre_result = pre_median.compute()

In [ ]:
plt.rcParams["figure.dpi"] = 300

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(8, 12))

bands = ["B12", "B8A", "B04", "B03", "B02"]
band_names = ["SWIR", "NIR", "Red", "Green", "Blue"]

contrast_stretch_percentile = (2, 98)
gamma = 1.8

for ax, band, name in zip(axes.flat, bands, band_names):
    data = normalisation_str_gm(pre_result[band].values, *contrast_stretch_percentile, gamma)

    ax.imshow(data, cmap="gray")
    ax.set_title(name)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
pre_red_norm = normalisation_str_gm(pre_result["B04"].values, *contrast_stretch_percentile, gamma)
pre_green_norm = normalisation_str_gm(pre_result["B03"].values, *contrast_stretch_percentile, gamma)
pre_blue_norm = normalisation_str_gm(pre_result["B02"].values, *contrast_stretch_percentile, gamma)
pre_nir_norm = normalisation_str_gm(pre_result["B8A"].values, *contrast_stretch_percentile, gamma)
pre_swir_norm = normalisation_str_gm(pre_result["B12"].values, *contrast_stretch_percentile, gamma)

In [ ]:
pre_true_color = np.dstack(
    [pre_red_norm, pre_green_norm, pre_blue_norm]
)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(pre_true_color)
ax.set_title("Pre-Fire True Color Composite: Gamma Adjusted")
ax.axis("off")

plt.tight_layout()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(pre_true_color))
ax.set_title("Pre-Fire True Color Composite: Equalized & Gamma Adjusted")
ax.axis("off")

plt.tight_layout()

In [ ]:
# Stack bands as RGB - access as dataset variables
pre_rgb = np.dstack(
    [pre_result["B04"].values, pre_result["B03"].values, pre_result["B02"].values]
)

# Normalize to 0-1 range
p2, p98 = np.nanpercentile(pre_rgb, (2, 98))
pre_rgb_normalized = np.clip((pre_rgb - p2) / (p98 - p2), 0, 1)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(pre_rgb_normalized)
ax.set_title("Pre-Fire True Color Composite")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(pre_rgb_normalized))
ax.set_title("Pre-Fire True Color Composite: Equalized")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
pre_false_color = np.dstack(
    [pre_swir_norm, pre_nir_norm, pre_blue_norm]
)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(pre_false_color)
ax.set_title("Pre-Fire False Color Composite: Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(pre_false_color))
ax.set_title("Pre-Fire False Color Composite: Equalized & Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
pre_snb = np.dstack(
    [pre_result["B12"].values, pre_result["B8A"].values, pre_result["B02"].values]
)

# Normalize to 0-1 range
p2, p98 = np.nanpercentile(pre_snb, (2, 98))
pre_snb_normalized = np.clip((pre_snb - p2) / (p98 - p2), 0, 1)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(pre_snb_normalized)
ax.set_title("Pre-Fire False Color Composite")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(pre_snb_normalized))
ax.set_title("Pre-Fire False Color Composite: Equalized")
ax.axis("off")

plt.tight_layout()
plt.show()

#### Part 2: Post-Fire Comparison

6. Search for and load Sentinel-2 imagery from June 21, 2025 (after the fire).

7. Produce true color and false color composites for the post-fire date using the same methods as Part 1.

8. Compare your pre-fire and post-fire false color composites. Where is the burn scar visible? How does the spectral signature of burned area differ from healthy vegetation in this band combination?


In [ ]:
# Create cloud mask from scl.
# Create cloud mask from scl.
# Mask is True where pixels are CLEAR.
scl = post_data["SCL"].squeeze()
scl_mask = scl.isin([2, 4, 5, 6, 7, 11])

post_data_masked = post_data[["B12", "B8A", "B04", "B03", "B02"]].where(scl_mask)

# Median now ignores NaN (SCL) pixels.
post_median = post_data_masked.median(dim = "time")
post_result = post_median.compute()

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(8, 12))

for ax, band, name in zip(axes.flat, bands, band_names):
    data = normalisation_str_gm(post_result[band].values, *contrast_stretch_percentile, gamma)

    ax.imshow(data, cmap="gray")
    ax.set_title(name)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
post_red_norm = normalisation_str_gm(post_result["B04"].values, *contrast_stretch_percentile, gamma)
post_green_norm = normalisation_str_gm(post_result["B03"].values, *contrast_stretch_percentile, gamma)
post_blue_norm = normalisation_str_gm(post_result["B02"].values, *contrast_stretch_percentile, gamma)
post_nir_norm = normalisation_str_gm(post_result["B8A"].values, *contrast_stretch_percentile, gamma)
post_swir_norm = normalisation_str_gm(post_result["B12"].values, *contrast_stretch_percentile, gamma)

In [ ]:
post_true_color = np.dstack(
    [post_red_norm, post_green_norm, post_blue_norm]
)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(post_true_color)
ax.set_title("Post-Fire True Color Composite: Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(post_true_color))
ax.set_title("Post-Fire True Color Composite: Equalized & Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
post_rgb = np.dstack(
    [post_result["B04"].values, post_result["B03"].values, post_result["B02"].values]
)

# Normalize to 0-1 range
p2, p98 = np.nanpercentile(post_rgb, (2, 98))
post_rgb_normalized = np.clip((post_rgb - p2) / (p98 - p2), 0, 1)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(post_rgb_normalized)
ax.set_title("Post-Fire True Color Composite")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(post_rgb_normalized))
ax.set_title("Post-Fire True Color Composite: Equalized")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
post_false_color = np.dstack(
    [post_swir_norm, post_nir_norm, post_blue_norm]
)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(post_false_color)
ax.set_title("Post-Fire False Color Composite: Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(post_false_color))
ax.set_title("Post-Fire False Color Composite: Equalized & Gamma Adjusted")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Stack bands as RGB - access as dataset variables
post_snb = np.dstack(
    [post_result["B12"].values, post_result["B8A"].values, post_result["B02"].values]
)

# Normalize to 0-1 range
p2, p98 = np.nanpercentile(post_snb, (2, 98))
post_snb_normalized = np.clip((post_snb - p2) / (p98 - p2), 0, 1)

# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(post_snb_normalized)
ax.set_title("Post-Fire False Color Composite")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(exposure.equalize_adapthist(post_snb_normalized))
ax.set_title("Post-Fire False Color Composite: Equalized")
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(8, 2, figsize=(12, 36))

images_left = [
    pre_true_color,
    exposure.equalize_adapthist(pre_true_color),
    pre_rgb_normalized,
    exposure.equalize_adapthist(pre_rgb_normalized),
    pre_false_color,
    exposure.equalize_adapthist(pre_false_color),
    pre_snb_normalized,
    exposure.equalize_adapthist(pre_snb_normalized)
]

images_right = [
    post_true_color,
    exposure.equalize_adapthist(post_true_color),
    post_rgb_normalized,
    exposure.equalize_adapthist(post_rgb_normalized),
    post_false_color,
    exposure.equalize_adapthist(post_false_color),
    post_snb_normalized,
    exposure.equalize_adapthist(post_snb_normalized)
]

titles_left = [
    "Pre: True Color (Gamma)",
    "Pre: True Color (Eq + Gamma)",
    "Pre: True Color (Percentile)",
    "Pre: True Color (Eq + Percentile)",
    "Pre: False Color (Gamma)",
    "Pre: False Color (Eq + Gamma)",
    "Pre: False Color (Percentile)",
    "Pre: False Color (Eq + Percentile)"
]

titles_right = [t.replace("Pre:", "Post:") for t in titles_left]

for row in range(8):
    axes[row, 0].imshow(images_left[row])
    axes[row, 0].set_title(titles_left[row])
    axes[row, 0].axis("off")
    
    axes[row, 1].imshow(images_right[row])
    axes[row, 1].set_title(titles_right[row])
    axes[row, 1].axis("off")

plt.tight_layout()
plt.show()

#### Part 3: Burn Severity Assessment

9. Calculate the Normalized Burn Ratio (NBR) for your pre-fire image using the formula:

$$NBR = \frac{NIR - SWIR}{NIR + SWIR}$$

What bands does this use (specifically, which Sentinel-2 band numbers)? Why are NIR and SWIR sensitive to fire damage?

10. Calculate NBR for your post-fire image.

11. Calculate the differenced NBR (dNBR) by subtracting post-fire NBR from pre-fire NBR. Multiply by 1000 to match standard severity thresholds.

12. Using the EFFIS burn severity classification below, plot the severity map for the burn scar area. Which areas experienced the highest severity?

| Class | dNBR range (×1000) |
|-------|-------------------|
| Unburned / Regrowth | < 100 |
| Low severity | 100–270 |
| Moderate-low severity | 270–440 |
| Moderate-high severity | 440–660 |
| High severity | ≥ 660 |


In [ ]:
# Calculate NBR for pre-fire
# NBR = (NIR - SWIR) / (NIR + SWIR)
# NIR = B8A, SWIR = B12
pre_nir = pre_result["B8A"].values.astype(float)
pre_swir = pre_result["B12"].values.astype(float)
pre_nbr = np.where((pre_nir + pre_swir) != 0, (pre_nir - pre_swir) / (pre_nir + pre_swir), 0)

# Calculate NBR for post-fire
# NBR = (NIR - SWIR) / (NIR + SWIR)
# NIR = B8A, SWIR = B12
post_nir = post_result["B8A"].values.astype(float)
post_swir = post_result["B12"].values.astype(float)
post_nbr = np.where((post_nir + post_swir) != 0, (post_nir - post_swir) / (post_nir + post_swir), 0)

dnbr = (pre_nbr - post_nbr) * 1000

In [ ]:
dnbr

In [ ]:
from matplotlib.colors import BoundaryNorm, ListedColormap

# EFFIS severity classes using finite bounds.
# Values outside this range will be clipped to edge colors.
bounds = [-500, 100, 270, 440, 660, 1500]
colors = ["green", "yellow", "orange", "red", "darkred"]
cmap = ListedColormap(colors)
norm = BoundaryNorm(bounds, cmap.N, clip = True)

# Plot dNBR with severity classification.
fig, axes = plt.subplots(1, 3, figsize = (18, 6))

# Pre-fire NBR.
im1 = axes[0].imshow(pre_nbr, cmap = "RdYlGn", vmin = -1, vmax = 1)
axes[0].set_title("Pre-Fire NBR")
axes[0].axis("off")
plt.colorbar(im1, ax = axes[0], shrink = 0.7)

# Post-fire NBR.
im2 = axes[1].imshow(post_nbr, cmap = "RdYlGn", vmin = -1, vmax = 1)
axes[1].set_title("Post-Fire NBR")
axes[1].axis("off")
plt.colorbar(im2, ax = axes[1], shrink = 0.7)

# dNBR with severity classification.
im3 = axes[2].imshow(dnbr, cmap = cmap, norm = norm)
axes[2].set_title("Burn Severity (dNBR × 1000)")
axes[2].axis("off")

# Custom colorbar with severity labels at class midpoints.
cbar = plt.colorbar(im3, ax = axes[2], shrink = 0.7)
cbar.set_ticks([-200, 185, 355, 550, 1080])
cbar.set_ticklabels(["Unburned", "Low", "Mod-Low", "Mod-High", "High"])

plt.tight_layout()
plt.show()

#### Part 4: Time Series Analysis

Two snapshots tell you about change, but a time series tells you about recovery. In this section, you'll extend your analysis to track NBR over the period before and after the fire.

16. Query the EOPF catalog for all available Sentinel-2 L2A scenes over your AOI from May 2025 through July 2025. Calculate NBR for each scene. (You'll want to write a helper function or loop to do this efficiently. Make sure to apply cloud masking to each scene.)

17. Create a faceted plot (small multiples) showing NBR maps for each month, similar to the NDMI example in Project Pythia. This gives you a visual sense of how the landscape changes through time.

In [ ]:
datetime_range = "2025-05-01/2025-07-31"

search = catalog.search(
    bbox=bbox,
    datetime=datetime_range,
    collections="sentinel-2-l2a",
    query={"eo:cloud_cover": {"lt": 30}}
)
items = search.item_collection()
print(f"Found {len(items)} scenes")

In [ ]:
def calculate_nbr_for_scene(item, bbox):
    """
    Load a single scene, apply cloud mask, and compute NBR.
    
    Pseudocode:
    1. Load B8A (NIR) and B12 (SWIR) and SCL bands using stac_load
    2. Apply cloud mask using SCL (same bitwise logic you used before)
    3. Calculate NBR = (NIR - SWIR) / (NIR + SWIR)
    4. Return NBR as numpy array along with the scene date
    """
    # Load single item (note: pass [item] as a list)
    data = odc.stac.load(
        [item],
        bands=["B8A", "B12", "SCL"],
        bbox=bbox,
        chunks={"time": 1}
    )
    
    # Extract date from item
    scene_date = item.datetime.strftime("%Y-%m-%d")
    
    # Apply cloud mask (adapt your existing SCL mask logic)
    # Create cloud mask from scl.
    # Mask is True where pixels are CLEAR.
    scl = data["SCL"].squeeze()
    scl_mask = scl.isin([2, 4, 5, 6, 7, 11])

    # Apply mask to spectral bands (SCL become NaN).
    data_masked = data[["B12", "B8A"]].where(scl_mask)

    # weekly composites
    result = data_masked.resample(time="1W").mean().compute()

    # Compute mean NBR per weekly period within AOI
    nbr_mean_biweekly = result.mean(dim=["x", "y"])
    
    # Calculate NBR
    nbr = np.where((result["B8A"] + result["B12"]) != 0, (result["B8A"] - result["B12"]) / (result["B8A"] + result["B12"]), 0)
    
    return nbr, scene_date

In [ ]:
nbr_results = []

for item in items:
    try:
        nbr, date = calculate_nbr_for_scene(item, bbox)
        nbr_results.append({"date": date, "nbr": nbr})
        print(f"Processed: {date}")
    except Exception as e:
        print(f"Failed: {item.id} - {e}")

In [ ]:
# Plot NBR time series in a grid layout.

n_scenes = len(nbr_results)
n_cols = 4
n_rows = (n_scenes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize = (16, 4 * n_rows))
axes = axes.flatten()

for i, result in enumerate(nbr_results):
    # Remove the leading band dimension (1, H, W) -> (H, W).
    nbr_2d = result["nbr"].squeeze()
    axes[i].imshow(nbr_2d, cmap = "RdYlGn", vmin = -1, vmax = 1)
    axes[i].set_title(result["date"])
    axes[i].axis("off")

# Hide any unused subplot axes.
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
items = list(
    catalog.search(
        bbox=bbox,
        datetime="2025-05-01/2025-07-30",
        collections="sentinel-2-l2a",
        query={"eo:cloud_cover": {"lt": 30}},
    ).item_collection()
)

print(f"Found {len(items)} items")

# Load all scenes
data_ts = odc.stac.load(
    items,
    bands=["B8A", "B12", "SCL"],
    bbox=bbox,
    chunks={"time": 1},
)


# Calculate NBR for each timestep
def calc_nbr(nir, swir):
    nir = nir.astype(float)
    swir = swir.astype(float)
    return (nir - swir) / (nir + swir + 1e-10)


nbr_ts = calc_nbr(data_ts["B8A"], data_ts["B12"])

# Mask clouds
scl = data_ts["SCL"]
valid = scl.isin([2, 4, 5, 6, 7, 11])
nbr_ts = nbr_ts.where(valid)

# weekly composites
nbr_biweekly = nbr_ts.resample(time="1W").mean().compute()

# Compute mean NBR per weekly period within AOI
nbr_mean_biweekly = nbr_biweekly.mean(dim=["x", "y"])

In [ ]:
# Maps with tighter range
g = nbr_biweekly.plot.imshow(
    col="time",
    col_wrap=4,
    vmin=-0.3,
    vmax=0.6,
    cmap="RdYlGn",
    figsize=(14, 8),
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40}
)

for ax in g.axs.flat:
    ax.axis("off")

plt.suptitle("Weekly NBR Composites", y=1.02)
plt.show()